In [1]:
import pandas as pd
import re
import os.path
import spacy
from os import listdir
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
from __future__ import unicode_literals, print_function
from spacy.lang.en import English
import random

In [2]:

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = BytesIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)#codec=codec,
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    return text.decode("utf-8")

In [3]:
regex_str=r"nitric acid mass lost test (NAMLT)|NAMLT|ASTM G67-04|ASTM|VMP3|EC-Lab|electrode|saturated calomel electrode (SCE)|SCE|platinum mesh|potentiodynamic polarisation|Tafel extrapolation|Eclab|polarisation test|Hardness tests|CALPHAD|A300 device|PanPhaseDiagram|Pandat|backscattered electron images (BSE)|BSE|JEOL|gun-scanning|microscopy|X-ray|EDS|electron backscatter diffraction (EBSD)|EBSD|3D FEG|FEI Quanta|PECS|precision etching coating system (PECS TM )|precision etching coating system|transmission electron microscopy|Gatan precision ion polishing system|JMP|inductively coupled plasma atomic emission spectroscopy|transmission electron microscopy (TEM)|TEM|ICP-AES"


In [4]:
# #file_path='/home/mehroz/pdf_filed_2/Untitled Folder/Aluminum paper/1-s2.0-S135964541630828X-main.pdf'
# path='/home/mehroz/pdf_filed_2/Untitled Folder/Aluminum paper/'

def preprocess(path):
    

                
                
    nlp = spacy.load("en")
    regex_str=r"nitric acid mass lost test (NAMLT)|NAMLT|ASTM G67-04|ASTM|VMP3|EC-Lab|electrode|saturated calomel electrode (SCE)|SCE|platinum mesh|potentiodynamic polarisation|Tafel extrapolation|Eclab|polarisation test|Hardness tests|CALPHAD|A300 device|PanPhaseDiagram|Pandat|backscattered electron images (BSE)|BSE|JEOL|gun-scanning|microscopy|X-ray|EDS|electron backscatter diffraction (EBSD)|EBSD|3D FEG|FEI Quanta|PECS|precision etching coating system (PECS TM )|precision etching coating system|transmission electron microscopy|Gatan precision ion polishing system|JMP|inductively coupled plasma atomic emission spectroscopy|transmission electron microscopy (TEM)|TEM|ICP-AES"
    sentencizer = nlp.create_pipe("sentencizer")
    nlp.add_pipe(sentencizer,first=True)#
    pdf_files=listdir(path)
    compositions_list=[]
    
    
            
#     for indx in range(0,len(partial_list)):
#         remaining_sent=partial_list[indx]
#         x1 = re.finditer(partial_regex,remaining_sent)
#         if x1:
            
#             for match_data in x1:
#                 result=match_data.span()
#                 print('\n remaining matched->',remaining_sent[result[0]:result[1]])
#                 #print('\n',sentence[result[0]:result[1]],'<-sentence-> ',sentence)
#                 r1=(remaining_sent.decode("utf-8"),{"entities":[(remaining_sent[0],remaining_sent[1],"Composition")]})
#                 #print('\n r type :',type(r))
#                 compositions_list.append(r1)
                
    for pdf_file_index in range(0,len(pdf_files)):
        pdf_file_path=path+pdf_files[pdf_file_index]
        print('\n Address :',pdf_file_path)
        
        text_file=convert_pdf_to_txt(pdf_file_path)
        doc = nlp(text_file)
        
        for sent in doc.sents:
            #print('\n ----- ',sent)
            #print('\n --- ',str(sent))
            #print('\n type :',type(sent))
            sentence=str(sent)
            x = re.finditer(regex_str,sentence)
            if x:
                #print('\n ----- ',sent)
                entity="entities"
                #entity=entity.encode("utf-8")
                category="method"
                #product=product.decode("utf-8")
                
                for match in x:
                    result=match.span()
                    print('\n matched->',sentence[result[0]:result[1]])
                    print('\n',sentence[result[0]:result[1]],'<-sentence-> ',sentence)
                    r=(sentence,{entity:[(result[0],result[1],category)]})
                    #print('\n r type :',type(r))
                    compositions_list.append(r)
                    

            
        
        
    
    
    return compositions_list                    
        #get_compositions(text_file)


In [5]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    spacy.prefer_gpu()
    spacy.require_gpu()
    
    nlp = spacy.blank('en')  # create blank Language class
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                
                    
                    
                print('\n Text is ',text)
                print('\n annotation is ',annotations)
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.3,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print('\n loses ',losses)
    return nlp
iterations=20


In [ ]:
# #path='/home/mehroz/pdf_filed_2/Untitled Folder/Aluminum paper/'
# #path='/home/mehroz/pdf_filed_2/Untitled_Folder/train_model/'
path='/home/cloud4/Mehroz/Mat_Sci/train_data/'

#compositions=preprocess(path)
#print(len(compositions))


In [10]:
print(len(compositions))
print('\n done')

3172

 done


In [ ]:
iterations=10
#spacy.prefer_gpu()
#prdnlp=train_spacy(compositions,iterations)

In [9]:
# # # # Save our trained Model
# modelfile = "Methods_GPU_model_0.1"
# prdnlp.to_disk(modelfile)


In [3]:
process_model = spacy.load("Methods_GPU_model_0.1")


In [4]:
nlp = spacy.load("en")

#nlp = spacy.load("test_model")
file_path='../one_paper_test/1-s2.0-S0010938X18319164-main.pdf'
test_text_file=convert_pdf_to_txt(file_path)

method
doc = nlp(test_text_file)
for sent in doc.sents:
    sentence=sent.text
    doc_processes = process_model(sentence)



    print('testing')
    for ent in doc_processes.ents:
        print('\n ==Start==')
        print('\n Method :',ent.text)
        #print('\n sentence :',sent)
        print('\n ==end==')



 ==Start==

 Method : TEM

 ==end==

 ==Start==

 Method : NAMLT

 ==end==

 ==Start==

 Method : yNd

 ==end==

 ==Start==

 Method : ICP-AES

 ==end==

 ==Start==

 Method : ASTM

 ==end==

 ==Start==

 Method : NAMLT

 ==end==

 ==Start==

 Method : VMP3

 ==end==

 ==Start==

 Method : electrode

 ==end==

 ==Start==

 Method : Tafel extrapolation

 ==end==

 ==Start==

 Method : CALPHAD

 ==end==

 ==Start==

 Method : Pandat

 ==end==

 ==Start==

 Method : PanPhaseDiagram

 ==end==

 ==Start==

 Method : BSE

 ==end==

 ==Start==

 Method : JEOL

 ==end==

 ==Start==

 Method : X-ray

 ==end==

 ==Start==

 Method : EDS

 ==end==

 ==Start==

 Method : EBSD

 ==end==

 ==Start==

 Method : EBSD

 ==end==

 ==Start==

 Method : PECS

 ==end==

 ==Start==

 Method : transmission electron microscopy

 ==end==

 ==Start==

 Method : TEM

 ==end==

 ==Start==

 Method : EDS

 ==end==

 ==Start==

 Method : TEM

 ==end==

 ==Start==

 Method : TEM

 ==end==

 ==Start==

 Method : JMP